In [2]:
import antares

ModuleNotFoundError: No module named 'antares'

In [1]:
__nbid__ = '0062'
__author__ = 'Carl Stubens, Sebastián Vicencio'
__edited__ = 'Gautham Narayan, Chien-Hsiu Lee, ANTARES Team <antares@noirlab.edu>'
__version__ = '20240601' # yyyymmdd
__datasets__ = ['']
__keywords__ = ['ANTARES', 'transient']

# ANTARES Filter Development Kit

_Carl Stubens, Gautham Narayan, Chien-Hsiu Lee, Sebastián Vicencio, ANTARES Team._

_Many thanks to Mike Fitzpatrick, Adam Scott, Knut Olsen, Jennifer Andrews, Robert Nikutta, Alice Jacques._

## Summary

This notebook demonstrates how to write filters for [ANTARES](https://antares.noirlab.edu) and test them against a sample of real data from [ZTF](https://ztf.caltech.edu/).

This notebook is intended to be used in Astro Data Lab's Jupyter environment. There, you will have access to ANTARES data. If you're not running in Data Lab, [sign up for Data Lab](https://datalab.noirlab.edu), then [log in to the notebook server](https://datalab.noirlab.edu/devbooks).

For new Data Lab accounts, this notebook will be automatically included in your `notebooks/` directory. Otherwise, you can save this `.ipynb` notebook file locally, and then upload it to your Data Lab Jupyter notebook server (use the 'Upload' button in the upper left corner).

In Data Lab, you MUST use the Kernel version "Python 3 (ANTARES)".

## Goals

To demonstrate:

1. How to write filters using the ANTARES filter API.
1. How to test filters against a small test dataset.


<a class="anchor" id="attribution"></a>
## Disclaimer & attribution

Disclaimers
-----------
Note that using the Astro Data Lab constitutes your agreement with our minimal [Disclaimers](https://datalab.noirlab.edu/disclaimers.php).

Acknowledgments
---------------
If you use **Astro Data Lab** in your published research, please include the text in your paper's Acknowledgments section:

_This research uses services or data provided by the Astro Data Lab, which is part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

If you use **SPARCL jointly with the Astro Data Lab platform** (via JupyterLab, command-line, or web interface) in your published research, please include this text below in your paper's Acknowledgments section:

_This research uses services or data provided by the SPectra Analysis and Retrievable Catalog Lab (SPARCL) and the Astro Data Lab, which are both part of the Community Science and Data Center (CSDC) Program of NSF NOIRLab. NOIRLab is operated by the Association of Universities for Research in Astronomy (AURA), Inc. under a cooperative agreement with the U.S. National Science Foundation._

In either case **please cite the following papers**:

* Data Lab concept paper: Fitzpatrick et al., "The NOAO Data Laboratory: a conceptual overview", SPIE, 9149, 2014, https://doi.org/10.1117/12.2057445

* Astro Data Lab overview: Nikutta et al., "Data Lab - A Community Science Platform", Astronomy and Computing, 33, 2020, https://doi.org/10.1016/j.ascom.2020.100411

If you are referring to the Data Lab JupyterLab / Jupyter Notebooks, cite:

* Juneau et al., "Jupyter-Enabled Astrophysical Analysis Using Data-Proximate Computing Platforms", CiSE, 23, 15, 2021, https://doi.org/10.1109/MCSE.2021.3057097

If publishing in a AAS journal, also add the keyword: `\facility{Astro Data Lab}`

And if you are using SPARCL, please also add `\software{SPARCL}` and cite:

* Juneau et al., "SPARCL: SPectra Analysis and Retrievable Catalog Lab", Conference Proceedings for ADASS XXXIII, 2024
https://doi.org/10.48550/arXiv.2401.05576

The NOIRLab Library maintains [lists of proper acknowledgments](https://noirlab.edu/science/about/scientific-acknowledgments) to use when publishing papers using the Lab's facilities, data, or services.


## Table of Contents

* [0. Background information on ANTARES](#background)
* [1. Initialize the dev kit](#connect)
* [2. Write a filter](#write)
 * [2.1 Hello world](#write-one)
 * [2.2 Example of a real filter](#write-two)
 * [2.3 Structure of a filter](#write-three)
* [3. Test a filter](#test)
 * [3.1 Constructing locus objects](#test-one)
* [4. Upload and use data files](#data)
 * [4.1 Uploading files into ANTARES](#data-one)
 * [4.2 Accessing files from filters](#data-two)
 * [4.3 Requesting files copy](#data-three)
* [5. Submit filter to ANTARES](#submit)

<a class="anchor" id="background"></a>
## 0. Background information on ANTARES

ANTARES receives alerts from surveys in real-time and sends them through a processing pipeline. The pipeline contains the following stages:

1. Associate the alert with the nearest point of known past measurements within a 1" radius. We call this a locus.
2. Discard alerts with a high probability of being false detections.
3. Discard alerts with poor image quality.
4. Look up associated objects in our catalogs.
5. If the alert's locus has two or more measurements on it, execute the filters.

The filters are Python functions which take a `LocusData` object as a single parameter. Functions on the `LocusData` provide access to the alert's properties, the data from past alerts on the locus, and the associated catalog objects. The `LocusData` also provides functions to set new properties on the alert, and to send it to output streams.

<a class="anchor" id="connect"></a>
## 1. Initialize the dev kit

This will configure the `antares` package to connect to the test database

In [4]:
# Imports
import antares.devkit as dk
dk.init()
# You should see a happy message that says that "ANTARES DevKit is ready!"

ModuleNotFoundError: No module named 'antares'

In [3]:
antares --version

NameError: name 'antares' is not defined